**ИНТЕРАКТИВНЫЙ ТРЕНАЖЕР ПО SQL. ОСНОВЫ РЕЛЯЦИОННОЙ МОДЕЛИ И SQL.**

[Курс](https://stepik.org/course/63054/syllabus) на Stepik.

Автор: **Озерова Галина Павловна**  
<img src="imgs/ozerova.jpg" title="Озерова Галина Павловна" width="200" height="200"/>

Конспектировал: **Илья Филимонов** (GiHub: [@IsFilimonov](https://github.com/IsFilimonov))  

<img src="imgs/IsFilimonov.png" title="Илья Филимонов" width="200" height="200"/>

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><strong>Введение</strong></a></span><ul class="toc-item"><li><span><a href="#От-конспектирующего" data-toc-modified-id="От-конспектирующего-1.1">От конспектирующего</a></span></li><li><span><a href="#Памятка-о-типичных-ошибках-и-способах-их-исправления" data-toc-modified-id="Памятка-о-типичных-ошибках-и-способах-их-исправления-1.2">Памятка о типичных ошибках и способах их исправления</a></span></li></ul></li><li><span><a href="#Основные-понятия-реляционных-баз-данных" data-toc-modified-id="Основные-понятия-реляционных-баз-данных-2"><strong>Основные понятия реляционных баз данных</strong></a></span><ul class="toc-item"><li><span><a href="#1.6-Таблица-&quot;Командировки&quot;,-запросы-на-выборку" data-toc-modified-id="1.6-Таблица-&quot;Командировки&quot;,-запросы-на-выборку-2.1"><strong>1.6 Таблица "Командировки", запросы на выборку</strong></a></span><ul class="toc-item"><li><span><a href="#Структура-и-наполнение-таблицы" data-toc-modified-id="Структура-и-наполнение-таблицы-2.1.1">Структура и наполнение таблицы</a></span></li></ul></li></ul></li></ul></div>

# **Введение**

## От конспектирующего
(Не реклама): сам курс использует диалект MySQL, но мне комфортнее использовать PostgreSQL 12.6 ([download](https://www.postgresql.org/download/)).  
[Рекомендации](https://wiki.postgresql.org/wiki/Don%27t_Do_This) того, чего не нужно делать в Postgres.

Для работы с базой лично я использую IDE [DataGrip](https://www.jetbrains.com/ru-ru/datagrip/) (платная) или [Dbeaver](https://dbeaver.io/) (бесплатная) - под MacOS. 

Это всё сторонние решения, но можно воспользоваться более [простым способом](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259) - использовать Jupyter Notebook в роли SQL IDE. Для этого нам потребуется установить несколько библиотек. Для работы с Jupyter Notebook я использую платформу [Anaconda](https://www.anaconda.com/), поэтому все нужные библиотеки для работы я буду устанавливать именно в эту среду. Я не буду останавливаться на деталях в вопросе "как?", чтобы в деталях не закопаться. В интернете полно подсказок. Главное, должны быть установлены:
- `ipython-sql` - [пакет](https://anaconda.org/conda-forge/ipython-sql) для работы с SQL запросами;
- `psycopg2` или `psycopg2-binary` - драйвер базы данных Postgres. Там есть нюансы с установкой;
- `sqlalchemy` - ORM для работы с SQL БД.


Более подробно читайте [статью](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259), которая была выше.

Для установки соединения с БД мы единожды используем SQL Alchemy.

In [1]:
import sqlalchemy

Далее создаем соединение.

In [2]:
sqlalchemy.create_engine("postgresql://localhost:5432/postgres")

Engine(postgresql://localhost:5432/postgres)

Загружаем extension.

In [3]:
%load_ext sql

Проверяем соединение.

In [4]:
%sql postgresql://localhost:5432/postgres

'Connected: @postgres'

Чтобы узнать какая версия установлена у вас, введите в консоле запроса БД (не в командной строке, а в ячейке):

In [5]:
%%sql

SELECT version();

 * postgresql://localhost:5432/postgres
1 rows affected.


version
"PostgreSQL 12.6 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit"


Вероятнее всего у вас уже создана БД `postgres`. Будем работать с ней. В БД нам нужно создать схему `stepik`, так сказать группировку (семантическую) для нужных нам таблиц. 

`DROP SCHEMA ... CASCADE` удаляет схему со всеми данными внутри.  
`IF NOT EXISTS` означает не делать ничего (только выдать замечание), если схема с таким именем уже существует.

In [6]:
%%sql
DROP SCHEMA IF EXISTS stepik CASCADE;

CREATE SCHEMA IF NOT EXISTS stepik;

 * postgresql://localhost:5432/postgres
Done.
Done.


[]

## Памятка о типичных ошибках и способах их исправления

1. Приведите синтаксис запроса к общепринятому:
    - если у вас есть время, стоит изучить [руководство по стилю SQL](https://www.sqlstyle.guide/ru/);
    - можете отформатировать ваш запрос с помощью сторонних [сервисов](https://codebeautify.org/sqlformatter).
2. Проверьте, что **ключевые слова**, **названия столбцов** и **значения в ячейках**, которые необходимо найти, написаны правильно. Особенно обратите внимание, чтобы в русских названиях столбцов не было английских букв.
3. Проверьте, что:
    - количество открывающихся скобок равно количеству закрывающихся;
    - запятые разделяют перечисление столбцов, но не ключевые слова;
    - запросы разделяются точкой с запятой.
4. Последовательность написания команд SQL:
```SQL
SELECT 'столбцы или * для выбора всех столбцов; обязательно'
FROM 'таблица; обязательно'
WHERE 'условие/фильтрация, например, city = 'Moscow'; необязательно'
GROUP BY 'столбец, по которому хотим сгруппировать данные; необязательно'
HAVING 'условие/фильтрация на уровне сгруппированных данных; необязательно'
ORDER BY 'столбец, по которому хотим отсортировать вывод; необязательно'
```
5. Если запрос включает подзапросы, выполните сначала подзапросы и удостоверьтесь, что получаете ожидаемый результат.
6. **Важно!** На платформе Stepik используется MySQL, версия 8.0.21.
7. Последовательность выполнения команд SQL:
<center><img src="imgs/sql-order.jpeg" title="SQL order" width="30%" height="30%"/></center>
8. Или другой вариант от [@Ken Flerlage](https://www.flerlagetwins.com/) со сравнением ([источник](https://www.flerlagetwins.com/2018/10/sql-part4.html)):
<center><img src="imgs/sql-order-operations.png" title="SQL order operations" width="50%" height="50%"/></center>
9. Специфические отличия MySQL от PostgreSQL:

[MySQL](https://www.mysqltutorial.org/mysql-having.aspx/): `FROM` => `WHERE` = `SELECT` = `GROUP BY` = `HAVING` = `ORDER BY` = `LIMIT`.   

[PostgreSQL](https://www.postgresqltutorial.com/postgresql-having/): `FROM` => `WHERE` = `GROUP BY` = `HAVING` = `SELECT` = `DISTINCT` = `ORDER BY` = `LIMIT`. 

# **Основные понятия реляционных баз данных**

## **1.6 Таблица "Командировки", запросы на выборку**

### Структура и наполнение таблицы

Таблица `trip`, в которой представлена информация о командировках сотрудников некоторой организации (фамилия сотрудника, город, куда он ездил, размер суточных, даты первого и последнего дня командировки):

